<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Capsule_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

# Define the CapsuleLayer class
class CapsuleLayer(nn.Module):
    def __init__(self, in_units, num_capsules, dim_capsules):
        super(CapsuleLayer, self).__init__()
        self.num_capsules = num_capsules  # Number of capsules
        self.dim_capsules = dim_capsules  # Dimension of each capsule
        self.fc = nn.Linear(in_units, num_capsules * dim_capsules)  # Fully connected layer

    def forward(self, x):
        batch_size = x.size(0)  # Get the batch size
        x = self.fc(x)  # Pass input through fully connected layer
        x = x.view(batch_size, self.num_capsules, self.dim_capsules)  # Reshape output to [batch_size, num_capsules, dim_capsules]
        return x

# Example instantiation and forward pass
capsule_layer = CapsuleLayer(in_units=512, num_capsules=10, dim_capsules=16)
input_tensor = torch.randn(32, 512)  # Example input (batch_size=32, in_units=512)
capsules = capsule_layer(input_tensor)

# Print the shape of the output capsules
print(capsules.shape)  # Expected shape: [32, 10, 16]